In [1]:
import numpy as np
import pandas as pd
import requests
from datetime import datetime, timedelta
from time import sleep
import regex

In [2]:
# EXTRACCIÓN 10 AÑOS


df= pd.read_csv("2022.csv", index_col=0)
df1= pd.read_csv("2023.csv", index_col=0)
df2= pd.read_csv("2024.csv", index_col=0)

#Por problemas con la api, el resto de años fueron facilitados por el profesor Daniel Tümmler
df3= pd.read_csv("2013.csv", index_col=0).reset_index()
df4= pd.read_csv("2014.csv", index_col=0).reset_index()
df5= pd.read_csv("2015.csv", index_col=0).reset_index()
df6= pd.read_csv("2016.csv", index_col=0).reset_index()
df7= pd.read_csv("2017.csv", index_col=0).reset_index()
df8= pd.read_csv("2018.csv", index_col=0).reset_index()
df9= pd.read_csv("2019.csv", index_col=0).reset_index()
df10= pd.read_csv("2020.csv", index_col=0).reset_index()
df11= pd.read_csv("2021.csv", index_col=0).reset_index()


In [3]:
df.columns

Index(['fecha', 'indicativo', 'nombre', 'provincia', 'altitud', 'tmed', 'prec',
       'tmin', 'horatmin', 'tmax', 'horatmax', 'dir', 'velmedia', 'racha',
       'horaracha', 'presMax', 'horaPresMax', 'presMin', 'horaPresMin',
       'hrMedia', 'hrMax', 'horaHrMax', 'hrMin', 'horaHrMin', 'sol'],
      dtype='object')

In [4]:
#Reorganización del orden de las columnas de los dataframes.

df, df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11= 
df, df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11[["fecha",
"indicativo","nombre","provincia", "altitud",
"tmed", "prec", "tmin", "horatmin", "tmax", 
"horatmax", "dir", "velmedia", "racha", "horaracha",
"presMax", 'horaPresMax', 'presMin', 'horaPresMin',
  "hrMedia", "hrMax", "horaHrMax", "hrMin", "horaHrMin", "sol"]]

In [6]:
#Se crea el dataframe total.

df_vc = pd.concat([df3, df4, df5, df6, df7, df8, df9, df10, df11, df, df1, df2], ignore_index=True)

In [7]:
df_vc = df_vc.drop_duplicates().reset_index(drop = True)
df_vc.shape

(3691105, 25)

In [9]:
#Eliminamos columnas que no nos interesan.

df_vc.drop(["presMax","horaPresMax","presMin","horaPresMin","sol"], axis = 1, inplace = True)

In [10]:
#Creamos nuevas tablas a partir de las tres columnas: indicativo, nombre y provincia.
indicativo_map = {indicativo : v for v, indicativo in enumerate(df_vc["indicativo"].unique(), start = 1)}

nombre_map = {nombre : v for v, nombre in enumerate(df_vc["nombre"].unique(), start = 1)}

provincia_map = {provincia : v for v, provincia in enumerate(df_vc["provincia"].unique(), start = 1)}

In [11]:
df_vc["indicativo_id"] = df_vc["indicativo"].map(indicativo_map)

df_vc["nombre_id"] = df_vc["nombre"].map(nombre_map)

df_vc["provincia_id"] = df_vc["provincia"].map(provincia_map)

In [12]:
df_indicativo = pd.DataFrame(indicativo_map.items(), columns = ["indicativo", "indicativo_id"])
df_indicativo = df_indicativo[["indicativo_id", "indicativo"]]

In [13]:
df_nombre = pd.DataFrame(nombre_map.items(), columns = ["nombre", "nombre_id"])
df_nombre = df_nombre[["nombre_id", "nombre"]]

In [14]:
df_provincia = pd.DataFrame(provincia_map.items(), columns = ["provincia", "provincia_id"])
df_provincia = df_provincia[["provincia_id", "provincia"]]

In [15]:
#Eliminamos de la tabla principal las columnas con los valores que ya están en otras tablas.

df_vc.drop(["indicativo", "nombre", "provincia"], axis = 1, inplace = True)

In [16]:
pd.merge(left = df_vc, right = df_indicativo, on = "indicativo_id")

,fecha,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,hrMedia,hrMax,...,hrMin,horaHrMin,dir,velmedia,racha,horaracha,indicativo_id,nombre_id,provincia_id,indicativo
0,2013-01-01,40,"8,1","1,4","4,5",23:20,"11,7",11:30,76.0,91.0,...,50.0,Varias,NaN,NaN,NaN,NaN,1,1,1,0255B
1,2013-01-01,410,"7,0","2,0","4,1",22:30,"9,8",13:00,91.0,94.0,...,85.0,01:10,36.0,"1,1","6,7",14:50,2,2,2,5612B
2,2013-01-01,808,"4,9","0,0","1,8",Varias,"8,0",15:50,88.0,93.0,...,74.0,16:00,30.0,"1,9","6,1",07:40,3,3,3,3094B
3,2013-01-01,582,"8,8","0,0","4,6",02:00,"13,0",13:50,65.0,88.0,...,47.0,13:00,28.0,"3,3","10,8",14:20,4,4,4,9394X
4,2013-01-01,17,"13,1","0,0","9,6",23:59,"16,6",15:20,67.0,80.0,...,56.0,14:30,23.0,"2,8","13,3",00:10,5,5,5,B434X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3691100,2024-11-29,371,"16,3","0,0","11,7",23:30,"20,9",13:59,69.0,87.0,...,54.0,14:00,19.0,"2,2","6,4",00:30,560,555,14,4436Y
3691101,2024-11-29,900,"21,2","0,0","18,0",20:52,"24,4",13:42,22.0,31.0,...,19.0,13:00,14.0,"4,4","11,7",09:30,319,315,10,C665T
3691102,2024-11-29,394,"15,6","0,0","11,0",23:00,"20,2",14:20,79.0,97.0,...,62.0,13:50,22.0,"1,1","6,1",16:30,237,234,29,3469A
3691103,2024-11-29,1135,"12,0","0,0","8,7",22:47,"15,4",14:37,62.0,86.0,...,27.0,00:00,24.0,"2,8","9,4",10:40,449,444,48,2140A


In [17]:
df_vc.head(10)

,fecha,altitud,tmed,prec,tmin,horatmin,tmax,horatmax,hrMedia,hrMax,horaHrMax,hrMin,horaHrMin,dir,velmedia,racha,horaracha,indicativo_id,nombre_id,provincia_id
0,2013-01-01,40,"8,1","1,4","4,5",23:20,"11,7",11:30,76.0,91.0,Varias,50.0,Varias,NaN,NaN,NaN,NaN,1,1,1
1,2013-01-01,410,"7,0","2,0","4,1",22:30,"9,8",13:00,91.0,94.0,Varias,85.0,01:10,36.0,"1,1","6,7",14:50,2,2,2
2,2013-01-01,808,"4,9","0,0","1,8",Varias,"8,0",15:50,88.0,93.0,Varias,74.0,16:00,30.0,"1,9","6,1",07:40,3,3,3
3,2013-01-01,582,"8,8","0,0","4,6",02:00,"13,0",13:50,65.0,88.0,09:10,47.0,13:00,28.0,"3,3","10,8",14:20,4,4,4
4,2013-01-01,17,"13,1","0,0","9,6",23:59,"16,6",15:20,67.0,80.0,Varias,56.0,14:30,23.0,"2,8","13,3",00:10,5,5,5
5,2013-01-01,88,"11,1","0,0","5,1",00:40,"17,1",14:00,56.0,79.0,06:20,40.0,13:20,24.0,"3,3","10,6",08:30,6,6,6
6,2013-01-01,715,"6,8","0,0","2,3",06:00,"11,3",13:50,71.0,93.0,Varias,53.0,13:50,33.0,"3,1","10,3",12:10,7,7,7
7,2013-01-01,874,"5,3","0,0","2,1",Varias,"8,5",14:20,85.0,99.0,Varias,64.0,14:50,26.0,"4,7","12,5",01:30,8,8,8
8,2013-01-01,43,"10,4","0,0","4,8",04:30,"16,0",13:50,67.0,81.0,00:30,47.0,13:50,28.0,"1,4","4,7",03:50,9,9,9
9,2013-01-01,25,"19,2","0,0","17,1",Varias,"21,4",14:35,64.0,78.0,Varias,54.0,01:04,3.0,"7,8","12,5",08:35,10,10,10


In [18]:
#Función para limpiar las columnas con fechas.

def clean_time(string):

    if string == "Varias" or pd.isna(string):
        return np.nan

    if string == "24:00":
        return "00:00"

    if string[0] not in ("0", "1", "2"):
        return np.nan
    if string[3] not in ("0", "1", "2", "3","4","5"):
        return np.nan

    return string

In [19]:
df_vc["horatmin"] = pd.to_datetime(df_vc["horatmin"].apply(lambda x : clean_time(x))).apply(lambda x : x.time() if not pd.isna(x) else x)
df_vc["horatmax"] = pd.to_datetime(df_vc["horatmax"].apply(lambda x : clean_time(x))).apply(lambda x : x.time() if not pd.isna(x) else x)
df_vc["horaracha"] = pd.to_datetime(df_vc["horaracha"].apply(lambda x : clean_time(x))).apply(lambda x : x.time() if not pd.isna(x) else x)
df_vc["horaHrMin"] = pd.to_datetime(df_vc["horaHrMin"].apply(lambda x : clean_time(x))).apply(lambda x : x.time() if not pd.isna(x) else x)
df_vc["horaHrMax"] = pd.to_datetime(df_vc["horaHrMax"].apply(lambda x : clean_time(x))).apply(lambda x : x.time() if not pd.isna(x) else x)

/var/folders/9m/h9bqsnt95t3bplz2w4d9fm4h0000gn/T/ipykernel_7303/2929415804.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_vc["horatmin"] = pd.to_datetime(df_vc["horatmin"].apply(lambda x : clean_time(x))).apply(lambda x : x.time() if not pd.isna(x) else x)
/var/folders/9m/h9bqsnt95t3bplz2w4d9fm4h0000gn/T/ipykernel_7303/2929415804.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_vc["horatmax"] = pd.to_datetime(df_vc["horatmax"].apply(lambda x : clean_time(x))).apply(lambda x : x.time() if not pd.isna(x) else x)
/var/folders/9m/h9bqsnt95t3bplz2w4d9fm4h0000gn/T/ipykernel_7303/2929415804.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`.

In [20]:
df_vc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3691105 entries, 0 to 3691104
Data columns (total 20 columns):
 #   Column         Dtype  
---  ------         -----  
 0   fecha          object 
 1   altitud        int64  
 2   tmed           object 
 3   prec           object 
 4   tmin           object 
 5   horatmin       object 
 6   tmax           object 
 7   horatmax       object 
 8   hrMedia        float64
 9   hrMax          float64
 10  horaHrMax      object 
 11  hrMin          float64
 12  horaHrMin      object 
 13  dir            float64
 14  velmedia       object 
 15  racha          object 
 16  horaracha      object 
 17  indicativo_id  int64  
 18  nombre_id      int64  
 19  provincia_id   int64  
dtypes: float64(4), int64(4), object(12)
memory usage: 563.2+ MB


In [21]:
#Función para convertir a columna float las columnas con valores numéricos.

def convertir_a_float(columna):
    # Reemplazar comas por puntos en la columna
    columna = columna.apply(lambda x: str(x).replace(',', '.') if isinstance(x, str) else x)
    # Intentar convertir a float, poniendo NaN en los valores no numéricos
    columna = pd.to_numeric(columna, errors='coerce')
    return columna
#"prec","tmin","horatmin","tmax","horatmax","velmedia","racha","horaracha","horaHrMax","horaHrMin"]].replace(str, np.nan).replace(",",".", regex=True).apply(pd.to_numeric, errors = "coerce")

In [22]:
df_vc["tmed"] = convertir_a_float(df_vc["tmed"])
df_vc["prec"] = convertir_a_float(df_vc["prec"])
df_vc["tmin"] = convertir_a_float(df_vc["tmin"])
df_vc["tmax"] = convertir_a_float(df_vc["tmax"])
df_vc["velmedia"] = convertir_a_float(df_vc["velmedia"])
df_vc["racha"] = convertir_a_float(df_vc["racha"])



In [23]:
df_vc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3691105 entries, 0 to 3691104
Data columns (total 20 columns):
 #   Column         Dtype  
---  ------         -----  
 0   fecha          object 
 1   altitud        int64  
 2   tmed           float64
 3   prec           float64
 4   tmin           float64
 5   horatmin       object 
 6   tmax           float64
 7   horatmax       object 
 8   hrMedia        float64
 9   hrMax          float64
 10  horaHrMax      object 
 11  hrMin          float64
 12  horaHrMin      object 
 13  dir            float64
 14  velmedia       float64
 15  racha          float64
 16  horaracha      object 
 17  indicativo_id  int64  
 18  nombre_id      int64  
 19  provincia_id   int64  
dtypes: float64(10), int64(4), object(6)
memory usage: 563.2+ MB


In [24]:
#Se reordenan las columnas del dataframe.

df_vc = df_vc[["fecha",
    "altitud",
    "tmed",
    "prec",
    "tmin",
    "horatmin",
    "tmax",
    "horatmax",
    "dir",
    "velmedia",
    "racha",
    "horaracha",
    "hrMedia",
    "hrMax",
    "horaHrMax",
    "hrMin",
    "horaHrMin",
    "indicativo_id",
    "nombre_id",
    "provincia_id"]] 

In [25]:
#Sacamos los dataframes limpios y organizados de la misma manera para su inserción en la base de datos.

df_22 = df_vc[df_vc["fecha"].between("2022-01-01", "2022-12-31")]
df_23 = df_vc[df_vc["fecha"].between("2023-01-01", "2023-12-31")]
df_24 = df_vc[df_vc["fecha"].between("2024-01-01", "2024-12-31")]
df_13 = df_vc[df_vc["fecha"].between("2013-01-01", "2013-12-31")]
df_14 = df_vc[df_vc["fecha"].between("2014-01-01", "2014-12-31")]
df_15 = df_vc[df_vc["fecha"].between("2015-01-01", "2015-12-31")]
df_16 = df_vc[df_vc["fecha"].between("2016-01-01", "2017-12-31")]
df_17 = df_vc[df_vc["fecha"].between("2017-01-01", "2017-12-31")]
df_18 = df_vc[df_vc["fecha"].between("2018-01-01", "2018-12-31")]
df_19 = df_vc[df_vc["fecha"].between("2019-01-01", "2019-12-31")]
df_20 = df_vc[df_vc["fecha"].between("2020-01-01", "2020-12-31")]
df_21 = df_vc[df_vc["fecha"].between("2021-01-01", "2021-12-31")]

In [26]:
df_vc['fecha']

0          2013-01-01
1          2013-01-01
2          2013-01-01
3          2013-01-01
4          2013-01-01
              ...    
3691100    2024-11-29
3691101    2024-11-29
3691102    2024-11-29
3691103    2024-11-29
3691104    2024-11-29
Name: fecha, Length: 3691105, dtype: object

In [27]:
df_13.to_csv("df_13.csv", index = False)
df_14.to_csv("df_14.csv", index = False)
df_15.to_csv("df_15.csv", index = False)
df_16.to_csv("df_16.csv", index = False)
df_17.to_csv("df_17.csv", index = False)
df_18.to_csv("df_18.csv", index = False)
df_19.to_csv("df_19.csv", index = False)
df_20.to_csv("df_20.csv", index = False)
df_21.to_csv("df_21.csv", index = False)
df_22.to_csv("df_22.csv", index = False)
df_23.to_csv("df_23.csv", index = False)
df_24.to_csv("df_24.csv", index = False)
df_provincia.to_csv("provincia.csv", index = False)
df_nombre.to_csv("ciudades.csv", index = False)
df_indicativo.to_csv("indicativo.csv", index = False)

In [28]:
#Ejemplo de formato y estructura de uno de los nuevos dataframes.

df_22.info()

<class 'pandas.core.frame.DataFrame'>
Index: 317963 entries, 2758010 to 3075972
Data columns (total 20 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   fecha          317963 non-null  object 
 1   altitud        317963 non-null  int64  
 2   tmed           312380 non-null  float64
 3   prec           308182 non-null  float64
 4   tmin           312475 non-null  float64
 5   horatmin       290588 non-null  object 
 6   tmax           312508 non-null  float64
 7   horatmax       302973 non-null  object 
 8   dir            253726 non-null  float64
 9   velmedia       254644 non-null  float64
 10  racha          253726 non-null  float64
 11  horaracha      238459 non-null  object 
 12  hrMedia        297711 non-null  float64
 13  hrMax          303820 non-null  float64
 14  horaHrMax      200998 non-null  object 
 15  hrMin          303840 non-null  float64
 16  horaHrMin      269595 non-null  object 
 17  indicativo_id  317963 non-n